In [13]:
import pandas as pd

submissions = pd.read_csv("sel_hn_stories.csv")

submissions.columns = ["submission_time", "upvotes", "url", "headline"]

submissions = submissions.dropna()
submissions.head(3)

,submission_time,upvotes,url,headline
0,2010-02-17T16:57:59Z,1,blog.jonasbandi.net,Software: Sadly we did adopt from the construc...
1,2014-02-04T02:36:30Z,1,blogs.wsj.com,Google’s Stock Split Means More Control for L...
2,2011-10-26T07:11:29Z,1,threatpost.com,SSL DOS attack tool released exploiting negoti...


In [47]:
tokenized_headlines = []
for item in submissions["headline"]:
    tokenized_headlines.append(item.split())

In [48]:
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
#tokenized_headlines=[[",",":","gG","google’s"]]
clean_tokenized = []
for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)   

clean_tokenized

[['software',
  'sadly',
  'we',
  'did',
  'adopt',
  'from',
  'the',
  'construction',
  'analogy'],
 ['googles',
  'stock',
  'split',
  'means',
  'more',
  'control',
  'for',
  'larry',
  'and',
  'sergey'],
 ['ssl',
  'dos',
  'attack',
  'tool',
  'released',
  'exploiting',
  'negotiation',
  'overhead'],
 ['immutability', 'and', 'blocks', 'lambdas', 'and', 'closures'],
 ['comment', 'optimiser', 'la', 'vitesse', 'de', 'wordpress'],
 ['ilk', 'is', 'not', 'as', 'good', 'for', 'you', 'as', 'you', 'think'],
 ['worldometers', '', 'real', 'time', 'world', 'statistics'],
 ['icrosoft', 'strikes', 'back', 'introduces', 'docs', 'for', 'facebook'],
 ['net', 'http', 'status', 'codes'],
 ['anecdata',
  'or',
  'how',
  'mckinseys',
  'story',
  'became',
  'sheryl',
  'sandbergs',
  'fact'],
 ['immigration', 'overhaul', 'passes', 'in', 'senate'],
 ['what', 'matters', 'most', 'at', 'adtech', 'sf', '2014'],
 ['amazon',
  'silk',
  'revisited',
  'is',
  'the',
  'split',
  'cloud',
  'brows

In [49]:
import numpy as np
unique_tokens = []
single_tokens = []

for token_list in clean_tokenized:
    for token in token_list:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)



In [54]:
counts=pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)
for i, item in enumerate(clean_tokenized):
    for token in item:
        if token in unique_tokens:
            counts.iloc[i][token] += 1


In [56]:
counts.head(5)

,and,for,as,you,is,the,split,good,how,what,...,frameworks,animated,walks,auctions,clouds,hammer,autonomous,vehicle,crowdsourcing,disaster
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
word_counts = counts.sum(axis=0)

counts = counts.loc[:,(word_counts >= 5) & (word_counts <= 100)]

In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, submissions["upvotes"], test_size=0.2, random_state=1)

#parameter to randomly select 20% of the rows for our test set,


In [63]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

In [64]:
submissions["upvotes"]

0         1
1         1
2         1
3        67
4         1
5         1
6         1
7        26
8         2
9         1
11        8
12        1
13        2
14        9
15      109
16       48
17        3
18        3
19        1
20       61
21        1
22        1
23       10
24       15
25        1
26        1
27        1
28        1
29        2
30        2
       ... 
2969      5
2970      2
2971      1
2972      3
2973      1
2974      1
2975      2
2976      1
2977      1
2978      1
2979      1
2980      9
2981     32
2982      1
2983      2
2984      1
2985    172
2986      2
2987      7
2988      7
2989      2
2990    170
2991      1
2992      1
2993      1
2994      1
2995     40
2996      2
2997      2
2998      1
Name: upvotes, Length: 2800, dtype: int64

In [66]:
mse = sum((predictions - y_test) ** 2) / len(predictions)
mse

2651.145705668968